In [0]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
    !pip install -q -U toai
    !pip install -q -U nb_black
    !pip install -q -U tensorflow-datasets
    !pip install -q -U --no-deps tensorflow-addons~=0.6
    print(__import__("toai").__version__)
    print(__import__("tensorflow").__version__)

In [1]:
from toai.imports import *
from toai.data import DataBundle, DataParams, DataContainer
from toai.image import ImageLearner, ImageAugmentor, ImageDataBundle, ImageResizer
import tensorflow as tf
from tensorflow import keras
#import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from toai.utils import download_file, unzip, save_file, load_file
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
!pip install tensorflow==2.0.0-rc0 

In [0]:
DATA_DIR = Path("data/web")
TEMP_DIR = Path("temp/web")
DATA_DIR.mkdir(parents=True, exist_ok=True)
TEMP_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
!kaggle competitions download -c demand-forecasting-kernels-only -p  {str(DATA_DIR)}

  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 41.2MB/s]
  0% 0.00/135k [00:00<?, ?B/s]
100% 135k/135k [00:00<00:00, 41.8MB/s]
  0% 0.00/3.08M [00:00<?, ?B/s]
100% 3.08M/3.08M [00:00<00:00, 102MB/s]


In [0]:
unzip.all_files_in_dir(DATA_DIR, ".zip")

In [0]:
train_df = pd.read_csv(DATA_DIR/'train.csv')

In [0]:
test_df = pd.read_csv(DATA_DIR/'test.csv')

In [15]:
test_df.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [14]:
train_df.head(10)

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
5,2013-01-06,1,1,12
6,2013-01-07,1,1,10
7,2013-01-08,1,1,9
8,2013-01-09,1,1,12
9,2013-01-10,1,1,9


In [15]:
train_df.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [17]:
train_df.shape

(913000, 4)

In [0]:
train_set = train_df.iloc[:,3:].values

In [25]:
train_set.shape

(913000, 1)

In [26]:
train_set

array([[13],
       [11],
       [14],
       ...,
       [74],
       [62],
       [82]])

In [0]:
scaler = MinMaxScaler(feature_range=(0,1))

In [0]:
train_set_norm = scaler.fit_transform(train_set)

In [7]:
train_set_norm

array([[0.05627706],
       [0.04761905],
       [0.06060606],
       ...,
       [0.32034632],
       [0.26839827],
       [0.35497835]])

In [0]:
x_train = []
y_train = []

In [0]:
for i in range(100, 913000):
    x_train.append(train_set_norm[i-100:i, 0])
    y_train.append(train_set_norm[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)

In [37]:
x_train

array([[0.05627706, 0.04761905, 0.06060606, ..., 0.08225108, 0.05627706,
        0.08225108],
       [0.04761905, 0.06060606, 0.05627706, ..., 0.05627706, 0.08225108,
        0.06926407],
       [0.06060606, 0.05627706, 0.04329004, ..., 0.08225108, 0.06926407,
        0.07792208],
       ...,
       [0.31168831, 0.38528139, 0.41991342, ..., 0.17748918, 0.27272727,
        0.25541126],
       [0.38528139, 0.41991342, 0.41991342, ..., 0.27272727, 0.25541126,
        0.32034632],
       [0.41991342, 0.41991342, 0.42857143, ..., 0.25541126, 0.32034632,
        0.26839827]])

In [38]:
y_train

array([0.06926407, 0.07792208, 0.07359307, ..., 0.32034632, 0.26839827,
       0.35497835])

In [0]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [14]:
x_train.shape

(912900, 100, 1)

In [0]:
series = x_train

In [0]:
def split_timeseries(
    series: np.ndarray, test_size: float
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    train_data, test_data = train_test_split(series, test_size=test_size, shuffle=False)
    val_data, test_data = train_test_split(test_data, test_size=0.4, shuffle=False)
    return train_data, val_data, test_data

In [0]:
def timeseries_extract_labels(
    series: np.ndarray, n_labels: int
) -> Tuple[np.ndarray, np.ndarray]:
    return series[:, :-n_labels], series[:, -n_labels:, 0]



In [0]:
train_data, val_data, test_data = split_timeseries(series, 0.4)

In [38]:
[series.shape for series in (train_data, val_data, test_data)]

[(547740, 100, 1), (219096, 100, 1), (146064, 100, 1)]

In [0]:
(train_data, train_labels), (val_data, val_labels), (test_data, test_labels) = [
    timeseries_extract_labels(series, 1) for series in (train_data, val_data, test_data)
]

In [44]:
[series.shape for series in (train_labels, val_labels, test_labels)]



[(912900, 1), (912900, 1), (912900, 1)]

In [0]:
model = keras.models.Sequential(
    [keras.layers.Flatten(input_shape=[99, 1]), keras.layers.Dense(1)]
)

In [0]:
model.compile(loss=keras.losses.mse, optimizer=keras.optimizers.Adam(lr=1e-3))



In [47]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 99)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 100       
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(
    train_data,
    train_labels,
    validation_data=(val_data, val_labels),
    epochs=2,
    verbose=2,
)



Train on 912900 samples, validate on 912900 samples
Epoch 1/2
912900/912900 - 77s - loss: 0.0016 - val_loss: 0.0014
Epoch 2/2
912900/912900 - 77s - loss: 0.0014 - val_loss: 0.0025


In [0]:
model = keras.models.Sequential(
    [keras.layers.LSTM(32, input_shape=[x_train.shape[1], 1]), keras.layers.Dense(1)]
)

In [0]:
model.compile(loss=keras.losses.mse, optimizer=keras.optimizers.Adam(lr=1e-3))

In [51]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(
    train_data,
    train_labels,
    validation_data=(val_data, val_labels),
    epochs=2,
    verbose=2,
)



In [0]:
class NaiveModel:
    def __init__(self, mode: str = "last"):
        self.mode = mode

    def predict(self, series: np.ndarray) -> np.ndarray:
        if self.mode == "mean":
            return np.mean(series, axis=1)
        return series[:, -1]

In [0]:
model = NaiveModel("last")

In [0]:
val_preds = model.predict(val_data)

In [56]:
val_preds, val_preds.shape

(array([[0.07359307],
        [0.08225108],
        [0.05627706],
        ...,
        [0.22077922],
        [0.17748918],
        [0.27272727]]), (912900, 1))

In [57]:
np.mean(keras.losses.mse(val_labels, val_preds))

0.003779783034790268

In [0]:
model = keras.models.Sequential(
    [
        keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[97, 1]),
        keras.layers.SimpleRNN(1),
    ]
)



In [0]:
model.compile(loss=keras.losses.mse, optimizer=keras.optimizers.Adam(lr=1e-3))

In [64]:
model.fit(
    train_data,
    train_labels,
    validation_data=(val_data, val_labels),
    epochs=2,
    verbose=2,
)



Train on 912900 samples, validate on 912900 samples
Epoch 1/2


KeyboardInterrupt: ignored

In [0]:
model = keras.models.Sequential(
    [
        keras.layers.Conv1D(128, kernel_size=10, padding="same", input_shape=[None, 1]),
        keras.layers.Conv1D(128, kernel_size=10, padding="same"),
        keras.layers.Conv1D(128, kernel_size=10, padding="same"),
        keras.layers.Conv1D(128, kernel_size=10, padding="same"),
        keras.layers.Dense(PRED_LENGTH),
    ]
)